In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import hvplot.pandas
import plotly.graph_objs as go
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pathlib import Path
import os
from datetime import datetime, timedelta
import pandas as pd
import mplfinance as mpf

In [ ]:
#pull data from yahoo finance
data =yf.download(tickers='TSLA', period='max', interval = '1d', start='2007-01-01')

In [ ]:
# Show Data
print(data)

In [ ]:
# build dataframe

TSLA_df = pd.DataFrame(data)
TSLA_df.head(10)

# change df to csv
TSLA_csv = TSLA_df.to_csv('/users/manu/desktop/project_2/tesla_csv', index=True)

#Show the df
TSLA_df.head(10)


In [ ]:
#Show Close Only
TSLA_close = TSLA_df.drop(columns =['Open','High','Low','Adj Close','Volume'])
TSLA_close.head(10)

In [ ]:
TSLA_returns = TSLA_close.pct_change().dropna()
TSLA_returns

In [ ]:
# Access to the News Api
newsapi = NewsApiClient(api_key='ce3e78ab07ed4649a677e4da44c06a82')

In [ ]:
# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

In [ ]:
# Define a function to build news df
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

In [ ]:
# Pull all the Tesla Stock headlines data from 2008 
TSLA_news_en = newsapi.get_everything(
    q=" TSLA AND Tesla AND 2008",
    language="en"
)

In [ ]:
# Show the total number of news
print(TSLA_news_en["totalResults"])

In [ ]:
# Create a DataFrame with the Tesla News
TSLA_en_df = create_df(TSLA_news_en["articles"], "en")
TSLA_en_df

In [ ]:
# Create CSV from News df
TSLA_en_df.to_csv('/users/manu/desktop/project_2/TSLA_csv', index=False, encoding='utf-8-sig')

In [ ]:
# define  sentiment analysis  
TSLA_sentiment = []
analyzer = SentimentIntensityAnalyzer()

for article in TSLA_news_en['articles']:
    try:
        date = article["publishedAt"][:10]
        text = article["content"][0:198]
        sentiment = analyzer.polarity_scores(text)
        pos = sentiment['pos']
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        compound = sentiment["compound"] 
        
        TSLA_sentiment.append({
            "text": text,
            "date": date,
            "positive": pos,
            "neutral": neu,
            "negative": neg,
            "compound": compound 
        })
        
    except AttributeError:
        pass

In [ ]:
# Build a SIA DF
TSLA_S_df = pd.DataFrame(TSLA_sentiment) 
cols = ['date', 'positive', 'neutral', 'negative', 'compound', 'text']
TSLA_S_df = TSLA_S_df[cols] 
TSLA_S_df.describe()

In [ ]:
#Display a candle stick chart
fig = go.Figure(data=[go.Candlestick(x=TSLA_df.index,
                open=TSLA_df['Open'],
                high=TSLA_df['High'],
                low=TSLA_df['Low'],
                close=TSLA_df['Close'])])

fig.show()